In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import torch.optim as optim
import BatchMaker

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import math
import PIL
import random
import time
import os
import cv2
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.5.0
Torchvision Version:  0.6.0


In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64))
    
        self.layer2 = nn.Sequential( 
            nn.Conv2d(64, 64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64))
        
        self.layer3 = nn.Sequential(
            nn.MaxPool2d(2, stride=2, padding=0))
            
        self.layer4 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))

        self.layer5 = nn.Sequential(
            nn.Conv2d(128, 128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))
            
        self.layer6 = nn.Sequential(
            nn.MaxPool2d(2, stride=2, padding=0))
            
        self.layer7 = nn.Sequential(
            nn.Conv2d(128, 256,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(256))
            
        self.layer8 = nn.Sequential(
            nn.Conv2d(256, 256,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(256))
            
        self.layer9 = nn.Sequential(
            nn.Conv2d(256, 256,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(256))
            
        self.layer10 = nn.UpsamplingBilinear2d(scale_factor=2)
            
        self.layer11 = nn.Sequential(
            nn.Conv2d(256, 128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))
            
        self.layer12 = nn.Sequential(
            nn.Conv2d(128, 128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))
            
        self.layer13 = nn.Sequential(
            nn.Conv2d(128, 128,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(128))
            
        self.layer14 = nn.UpsamplingBilinear2d(scale_factor=2)
            
        self.layer15 = nn.Sequential(
            nn.Conv2d(128, 64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            
        self.layer16 = nn.Sequential(
            nn.Conv2d(64, 64,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            
        self.layer17 = nn.Sequential(
            nn.Conv2d(64, 1,kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(),
            nn.BatchNorm2d(1))
        
        self.layer18 = nn.Softmax(dim=1)
            
            
    def forward(self,x):
        x = self.layer1(x)  
        x = self.layer2(x)  
        x = self.layer3(x)  
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)  
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x)  
        x = self.layer10(x)  
        x = self.layer11(x)  
        x = self.layer12(x)  
        x = self.layer13(x)  
        x = self.layer14(x)  
        x = self.layer15(x)  
        x = self.layer16(x)
        x = self.layer17(x) 
        x = x.view(1, -1)
        x = self.layer18(x)
        x = x.reshape(1, 180, 240)
        
        return x

In [3]:
# Check if GPU is available and move the model over to GPU
if torch.cuda.is_available():
    device = torch.device("cuda: 0")
    gpu_name = torch.cuda.get_device_name()
    print(f"Running on your {gpu_name} (GPU)")
else:
    device = torch.device("cpu")
    print("Running on your CPU")

net = CNN().to(device)

Running on your GeForce GTX 1080 (GPU)


In [4]:
Generator = BatchMaker.BatchMaker

batch_size = 1
training_file = "training.csv"
testing_file = "testing.csv"

In [5]:
def distance(output, target):
    
    center_output = np.where(output==np.amax(output))
    center_target = np.where(target==np.amax(target))
    
    x1 = center_output[1]
    y1 = center_output[0]
    x2 = center_target[1]
    y2 = center_target[0]
    
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

In [6]:
loss_fn = nn.L1Loss(reduction='sum')
optimizer = optim.Adam(net.parameters(), lr=0.001)

epochs = 1
epochs_no_improve_limit = 7

train_losses = []
val_losses = []


def train():
    epochs_no_improve = 0
    min_val_loss = np.Inf
    since = time.time()
    iteration = 0
    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch + 1, epochs))
        print('-' * 10)
        val_loss = 0
        train_loss = 0
        
        for train_inputs, train_labels in Generator(training_file, batch_size):
            optimizer.zero_grad()
            train_inputs = train_inputs.transpose(0,3,1,2)
#             print(train_inputs.shape)
            
            output = net(torch.from_numpy(train_inputs).to(device))
#             print(output.shape)
#             print(np.sum(output.cpu().detach().numpy()))
            
            train_labels = np.divide(train_labels, np.sum(train_labels))
#             print(np.sum(train_labels))

            
            loss = loss_fn(output.cpu(), torch.from_numpy(train_labels).cpu())*(distance(output.cpu().detach().numpy(), train_labels)/4)
            train_loss += loss
            
#             FileName = f'Data/Output/output-%04d.png' % iteration
            
#             cv2.imwrite(FileName, np.float32(output[0].cpu().detach().numpy()) )
            
#             fig = plt.figure(figsize = (32,24))
#             ax1 = fig.add_subplot(2, 2, 1)
#             ax1.set_title('Input red')
#             ax2 = fig.add_subplot(2, 2, 2)
#             ax2.set_title('Input green')
#             ax3 = fig.add_subplot(2, 2, 3)
#             ax3.set_title('Label')
#             ax4 = fig.add_subplot(2, 2, 4)
#             ax4.set_title('Output of the CNN')
            
#             sns.heatmap(train_inputs[0][2], vmin=0, vmax=1, cmap='gray', ax=ax1, cbar_kws={'shrink': .9})
#             sns.heatmap(train_inputs[0][2], vmin=0, vmax=1, ax=ax2, cbar_kws={'shrink': .9})
#             sns.heatmap((train_labels[0]), ax=ax3, cbar_kws={'shrink': .9})
#             sns.heatmap((output[0].cpu().detach().numpy()), ax=ax4, cbar_kws={'shrink': .9})
#             plt.show()


#             writer.add_scalar('Loss/train', loss, iteration)

            print('Training Loss: {:.4f}'.format(loss.item()))
            loss.backward()
            optimizer.step()
            iteration += 1
        
        train_loss = train_loss / len(dataloaders['train'])
        train_losses.append(float(train_loss))
        
        del train_inputs
        del train_labels
        del output
        del loss
        torch.cuda.empty_cache()
        
#         with torch.no_grad():
#             for val_inputs, val_labels in dataloaders['val']:
#                 torch.cuda.empty_cache()
#                 output = net(val_inputs.to(device)).reshape(batch_size, output_height, output_width)
#                 loss = loss_fn(output.cpu(), val_labels.cpu().reshape(batch_size, output_height, output_width))
#                 val_loss += loss

#             val_loss = val_loss / len(dataloaders['val'])
#             val_losses.append(float(val_loss))
#             print('-' * 10)
#             print('Validation Loss: {:.4f}'.format(val_loss))

#             if val_loss < min_val_loss:
#                 torch.save({'state_dict': net.state_dict()}, 'Nets/pt-labi_CNN.pt')
#                 epochs_no_improve = 0
#                 min_val_loss = val_loss
#             else:
#                 epochs_no_improve += 1
#                 if epochs_no_improve == epochs_no_improve_limit:
#                     print('Early stopping initiated')
#                     model = torch.load('Nets/pt-labi_CNN.pt')
#                     print('Best model so far has been loaded')
    print('Least validation Loss: {:4f}'.format(min_val_loss))
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Finished training')
train()

Epoch 1/1
----------
Training Loss: 19.9700
Training Loss: 45.8272
Training Loss: 6.9661
Training Loss: 18.3688
Training Loss: 51.0700
Training Loss: 1.7097
Training Loss: 0.4971
Training Loss: 0.9871
Training Loss: 1.1474
Training Loss: 11.4927
Training Loss: 1.4604
Training Loss: 1.5524
Training Loss: 34.9468
Training Loss: 0.4021
Training Loss: 19.4339
Training Loss: 8.6957
Training Loss: 0.9223
Training Loss: 35.3130
Training Loss: 5.9800
Training Loss: 31.6537
Training Loss: 3.3916
Training Loss: 5.4231
Training Loss: 32.2879
Training Loss: 2.3258
Training Loss: 17.9514
Training Loss: 23.9330
Training Loss: 47.2057
Training Loss: 2.7763
Training Loss: 3.3919
Training Loss: 50.7545
Training Loss: 2.8460
Training Loss: 58.9500
Training Loss: 0.0000
Training Loss: 1.8997
Training Loss: 1.2817
Training Loss: 9.2436
Training Loss: 30.8862
Training Loss: 58.7325
Training Loss: 0.2532
Training Loss: 0.2364
Training Loss: 2.2940
Training Loss: 0.4636
Training Loss: 0.3978
Training Loss: 1

Training Loss: 0.6297
Training Loss: 42.4651
Training Loss: 0.6955
Training Loss: 1.8484
Training Loss: 63.3979
Training Loss: 49.4697
Training Loss: 1.2152
Training Loss: 1.0779
Training Loss: 0.6477
Training Loss: 0.6779
Training Loss: 1.8988
Training Loss: 53.7423
Training Loss: 0.2021
Training Loss: 0.2711
Training Loss: 0.9633
Training Loss: 0.3382
Training Loss: 0.2598
Training Loss: 0.7020
Training Loss: 0.0000
Training Loss: 0.8247
Training Loss: 0.0000
Training Loss: 2.9669
Training Loss: 17.9387
Training Loss: 7.3468
Training Loss: 0.3833
Training Loss: 0.3890
Training Loss: 21.8923
Training Loss: 0.1255
Training Loss: 0.0000
Training Loss: 1.3420
Training Loss: 26.6629
Training Loss: 0.0000
Training Loss: 0.4190
Training Loss: 11.4013
Training Loss: 0.4811
Training Loss: 0.4072
Training Loss: 1.0852
Training Loss: 0.0000
Training Loss: 50.8197
Training Loss: 0.0000
Training Loss: 0.2281
Training Loss: 0.6837
Training Loss: 7.8994
Training Loss: 1.0637
Training Loss: 0.8339
T

Training Loss: 0.4301
Training Loss: 1.1248
Training Loss: 1.5541
Training Loss: 5.7707
Training Loss: 14.4839
Training Loss: 0.3870
Training Loss: 0.6786
Training Loss: 0.3342
Training Loss: 5.7557
Training Loss: 0.4022
Training Loss: 0.7321
Training Loss: 0.8134
Training Loss: 0.0000
Training Loss: 0.5789
Training Loss: 0.6464
Training Loss: 0.0824
Training Loss: 0.1321
Training Loss: 0.0000
Training Loss: 1.4272
Training Loss: 0.0935
Training Loss: 2.5665
Training Loss: 0.3112
Training Loss: 0.9950
Training Loss: 1.3359
Training Loss: 0.5797
Training Loss: 0.2151
Training Loss: 0.0000
Training Loss: 0.3557
Training Loss: 0.2356
Training Loss: 0.3030
Training Loss: 0.2742
Training Loss: 0.3292
Training Loss: 0.0000
Training Loss: 0.9107
Training Loss: 3.0504
Training Loss: 0.9613
Training Loss: 0.1234
Training Loss: 0.0994
Training Loss: 0.1497
Training Loss: 16.7392
Training Loss: 0.2848
Training Loss: 0.3478
Training Loss: 1.1206
Training Loss: 0.4684
Training Loss: 0.2388
Training

Training Loss: 0.1325
Training Loss: 2.1136
Training Loss: 0.1672
Training Loss: 1.1299
Training Loss: 0.0000
Training Loss: 13.5727
Training Loss: 0.3468
Training Loss: 0.0000
Training Loss: 0.3660
Training Loss: 1.8532
Training Loss: 0.3972
Training Loss: 1.2237
Training Loss: 40.0543
Training Loss: 0.1458
Training Loss: 0.0000
Training Loss: 0.1299
Training Loss: 0.0000
Training Loss: 0.0000
Training Loss: 0.9771
Training Loss: 0.1430
Training Loss: 0.8146
Training Loss: 0.2802
Training Loss: 0.1614
Training Loss: 0.1326
Training Loss: 0.5529
Training Loss: 0.4160
Training Loss: 19.8719
Training Loss: 1.0415
Training Loss: 0.3269
Training Loss: 1.7057
Training Loss: 2.7250
Training Loss: 0.3713
Training Loss: 0.1251
Training Loss: 2.1654
Training Loss: 0.3557
Training Loss: 0.8074
Training Loss: 0.4043
Training Loss: 0.0000
Training Loss: 0.5493
Training Loss: 0.1326
Training Loss: 0.2614
Training Loss: 0.2268
Training Loss: 0.0000
Training Loss: 9.0692
Training Loss: 0.6481
Trainin

Training Loss: 13.6269
Training Loss: 0.8635
Training Loss: 0.2653
Training Loss: 0.5041
Training Loss: 0.2995
Training Loss: 2.0692
Training Loss: 0.8112
Training Loss: 0.1744
Training Loss: 0.7926
Training Loss: 0.0000
Training Loss: 0.0000
Training Loss: 31.7629
Training Loss: 1.6736
Training Loss: 2.6799
Training Loss: 50.5002
Training Loss: 0.0000
Training Loss: 2.0560
Training Loss: 0.3548
Training Loss: 0.4510
Training Loss: 0.4515
Training Loss: 0.8970
Training Loss: 0.3172
Training Loss: 0.7294
Training Loss: 7.9026
Training Loss: 0.6998
Training Loss: 0.2722
Training Loss: 0.1706
Training Loss: 0.9463
Training Loss: 0.1510
Training Loss: 0.1296
Training Loss: 0.1458
Training Loss: 0.5689
Training Loss: 16.9912
Training Loss: 0.8632
Training Loss: 2.2971
Training Loss: 0.2904
Training Loss: 29.5633
Training Loss: 0.2663
Training Loss: 0.4182
Training Loss: 0.4757
Training Loss: 0.2991
Training Loss: 0.1287
Training Loss: 0.5000
Training Loss: 0.2788
Training Loss: 0.0000
Train

Training Loss: 0.5052
Training Loss: 0.0870
Training Loss: 0.0000
Training Loss: 0.4778
Training Loss: 0.1692
Training Loss: 0.0000
Training Loss: 0.0000
Training Loss: 0.0926
Training Loss: 0.2550
Training Loss: 1.9159
Training Loss: 0.0000
Training Loss: 0.0605
Training Loss: 0.5612
Training Loss: 0.0000
Training Loss: 0.0757
Training Loss: 0.0000
Training Loss: 0.0000
Training Loss: 0.3734
Training Loss: 0.1878
Training Loss: 0.6394
Training Loss: 1.5185
Training Loss: 0.1057
Training Loss: 0.7707
Training Loss: 0.7693
Training Loss: 1.6871
Training Loss: 0.0000
Training Loss: 1.4918
Training Loss: 0.0936
Training Loss: 0.4545
Training Loss: 0.0000
Training Loss: 0.0708
Training Loss: 0.7673
Training Loss: 0.0000
Training Loss: 0.1250
Training Loss: 0.0765
Training Loss: 0.0000
Training Loss: 0.5365
Training Loss: 0.2195
Training Loss: 0.1453
Training Loss: 0.1076
Training Loss: 0.6617
Training Loss: 0.6130
Training Loss: 0.1168
Training Loss: 0.1592
Training Loss: 0.1167
Training L

Training Loss: 0.0848
Training Loss: 0.2310
Training Loss: 0.0000
Training Loss: 0.0000
Training Loss: 0.7212
Training Loss: 0.1547
Training Loss: 0.1190
Training Loss: 0.8868
Training Loss: 3.1440
Training Loss: 0.0000
Training Loss: 0.0000
Training Loss: 0.7634
Training Loss: 0.0000
Training Loss: 1.6951
Training Loss: 0.0000
Training Loss: 0.0000
Training Loss: 0.1671
Training Loss: 0.1407
Training Loss: 0.3511
Training Loss: 0.3038
Training Loss: 0.1174
Training Loss: 0.1425
Training Loss: 2.4807
Training Loss: 0.1975
Training Loss: 0.2172
Training Loss: 2.2836
Training Loss: 1.7756
Training Loss: 0.8803
Training Loss: 0.5193
Training Loss: 0.3580
Training Loss: 2.4316
Training Loss: 0.4750
Training Loss: 0.1068
Training Loss: 0.1370
Training Loss: 0.1323
Training Loss: 0.3576
Training Loss: 0.3205
Training Loss: 0.3553
Training Loss: 0.0000
Training Loss: 0.1241
Training Loss: 0.5547
Training Loss: 1.2283
Training Loss: 0.1183
Training Loss: 0.1008
Training Loss: 0.2527
Training L

KeyboardInterrupt: 

In [18]:
output_width = 240
output_height = 180
def test():
    test_loss = 0
    iteration = 0
    num_test_samples = 640
    with torch.no_grad():
        for idx, (test_inputs, test_labels) in enumerate(Generator(testing_file, 1)):
            if idx >= num_test_samples:
                break
            else:
                output = net(torch.from_numpy(test_inputs.transpose(0,3,1,2)).to(device))
                loss = loss_fn(output.cpu(), torch.from_numpy(test_labels).cpu())*(distance(output.cpu().detach().numpy(), test_labels)/4)
                output = output[0].reshape(output_height, output_width).cpu().detach().numpy()
                test_labels = test_labels[0].reshape(output_height, output_width)
                test_loss += loss

                FileName = f'Data/Output/output-%04d.png' % iteration
                
                iteration += 1
            
                cv2.imwrite(FileName, np.float32(output) * 20000)
                
#                 fig = plt.figure(figsize = (32,24))
#                 ax1 = fig.add_subplot(2, 2, 1)
#                 ax1.set_title('Output')
#                 ax2 = fig.add_subplot(2, 2, 2)
#                 ax2.set_title('Label')

#                 sns.heatmap(output,ax=ax1, square=True, cbar_kws={'shrink': .8})
#                 sns.heatmap(test_labels, vmin=0, vmax=1, ax=ax2, square=True, cbar_kws={'shrink': .8})
#                 plt.show()
        
        test_loss = test_loss/num_test_samples
        print('Average test loss: ' ,test_loss.numpy())
        print('Testing completed')
test()

Average test loss:  41.30497
Testing completed
